In this lab exercise using KNN classier to predict survival of titanic passenger.


In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

plt.rcParams["figure.figsize"] = [8,6]
sns.set_style("darkgrid")
#import titatic data set
titanic_data = sns.load_dataset('titanic')
#return first 5 rows
titanic_data.head(30)

survived  pclass     sex   age  sibsp  parch      fare embarked   class  \
0          0       3    male  22.0      1      0    7.2500        S   Third   
1          1       1  female  38.0      1      0   71.2833        C   First   
2          1       3  female  26.0      0      0    7.9250        S   Third   
3          1       1  female  35.0      1      0   53.1000        S   First   
4          0       3    male  35.0      0      0    8.0500        S   Third   
5          0       3    male   NaN      0      0    8.4583        Q   Third   
6          0       1    male  54.0      0      0   51.8625        S   First   
7          0       3    male   2.0      3      1   21.0750        S   Third   
8          1       3  female  27.0      0      2   11.1333        S   Third   
9          1       2  female  14.0      1      0   30.0708        C  Second   
10         1       3  female   4.0      1      1   16.7000        S   Third   
11         1       1  female  58.0      0      0   26.5500        S   First   
12         0       3    male  20.0      0      0    8.0500        S   Third   
13         0       3    male  39.0      1      5   31.2750        S   Third   
14         0       3  female  14.0      0      0    7.8542        S   Third   
15         1       2  female  55.0      0      0   16.0000        S  Second   
16         0       3    male   2.0      4      1   29.1250        Q   Third   
17         1       2    male   NaN      0      0   13.0000        S  Second   
18         0       3  female  31.0      1      0   18.0000        S   Third   
19         1       3  female   NaN      0      0    7.2250        C   Third   
20         0       2    male  35.0      0      0   26.0000        S  Second   
21         1       2    male  34.0      0      0   13.0000        S  Second   
22         1       3  female  15.0      0      0    8.0292        Q   Third   
23         1       1    male  28.0      0      0   35.5000        S   First   
24         0       3  female   8.0      3      1   21.0750        S   Third   
25         1       3  female  38.0      1      5   31.3875        S   Third   
26         0       3    male   NaN      0      0    7.2250        C   Third   
27         0       1    male  19.0      3      2  263.0000        S   First   
28         1       3  female   NaN      0      0    7.8792        Q   Third   
29         0       3    male   NaN      0      0    7.8958        S   Third   

      who  adult_male deck  embark_town alive  alone  
0     man        True  NaN  Southampton    no  False  
1   woman       False    C    Cherbourg   yes  False  
2   woman       False  NaN  Southampton   yes   True  
3   woman       False    C  Southampton   yes  False  
4     man        True  NaN  Southampton    no   True  
5     man        True  NaN   Queenstown    no   True  
6     man        True    E  Southampton    no   True  
7   child       False  NaN  Southampton    no  False  
8   woman       False  NaN  Southampton   yes  False  
9   child       False  NaN    Cherbourg   yes  False  
10  child       False    G  Southampton   yes  False  
11  woman       False    C  Southampton   yes   True  
12    man        True  NaN  Southampton    no   True  
13    man        True  NaN  Southampton    no  False  
14  child       False  NaN  Southampton    no   True  
15  woman       False  NaN  Southampton   yes   True  
16  child       False  NaN   Queenstown    no  False  
17    man        True  NaN  Southampton   yes   True  
18  woman       False  NaN  Southampton    no  False  
19  woman       False  NaN    Cherbourg   yes   True  
20    man        True  NaN  Southampton    no   True  
21    man        True    D  Southampton   yes   True  
22  child       False  NaN   Queenstown   yes   True  
23    man        True    A  Southampton   yes   True  
24  child       False  NaN  Southampton    no  False  
25  woman       False  NaN  Southampton   yes  False  
26    man        True  NaN    Cherbourg    no   True  
27    man     

If you observe the coloum, some value of columns are numerical and some are categorical and not all column required to predict the survival of the passenger

Drop the unnecessary column and leave column, survived , pclass, sex, age and embarked. Replace columns sipsp and parch with "family" which combination of these two colums using following command

In [2]:
titanic_data["family"] = titanic_data ["sibsp"] + titanic_data["parch"] + 1

In [3]:
titanic_data = titanic_data[['survived', 'pclass', 'sex', 'age', 'embarked', 'family']]
titanic_data.head()

survived  pclass     sex   age embarked  family
0         0       3    male  22.0        S       2
1         1       1  female  38.0        C       2
2         1       3  female  26.0        S       1
3         1       1  female  35.0        S       2
4         0       3    male  35.0        S       1

Replace the categorical value with numerical and replace the missing value if any with the appropriate value ( see lecture on data preparation)

In [5]:
titanic_data.isnull().sum()

survived      0
pclass        0
sex           0
age         177
embarked      2
family        0
dtype: int64

In [6]:
# Replacing the missing value
titanic_data['age'].fillna(titanic_data['age'].median(), inplace=True)
titanic_data['embarked'].fillna(titanic_data['embarked'].mode()[0], inplace=True)

# Replacing the categorical value with numerical
titanic_data = pd.get_dummies(titanic_data, columns=['sex', 'embarked'])
titanic_data.head()

survived  pclass   age  family  sex_female  sex_male  embarked_C  \
0         0       3  22.0       2       False      True       False   
1         1       1  38.0       2        True     False        True   
2         1       3  26.0       1        True     False       False   
3         1       1  35.0       2        True     False       False   
4         0       3  35.0       1       False      True       False   

   embarked_Q  embarked_S  
0       False        True  
1       False       False  
2       False        True  
3       False        True  
4       False        True

Perform classifcation using KNN with several value of K and report the performance based on performance metric

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

X = titanic_data.drop('survived', axis=1)
y = titanic_data['survived']

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Defining a list of K values to try
k_values = [3, 5, 7, 9, 11]

# Initializing an empty dictionary to store results
results = {}

# Iterating over each K value
for k in k_values:
    # Training the KNN classifier
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    
    # Making predictions on the test set
    y_pred = knn.predict(X_test)
    
    # Evaluating the model using classification report
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Storing the performance metrics in the results dictionary
    results[k] = report

# Printing the performance metrics for each K value
for k, report in results.items():
    print(f"Performance metrics for K={k}:")
    print()
    print(classification_report(y_test, y_pred))
    print()

Performance metrics for K=3:

              precision    recall  f1-score   support

           0       0.79      0.90      0.84       105
           1       0.83      0.65      0.73        74

    accuracy                           0.80       179
   macro avg       0.81      0.78      0.78       179
weighted avg       0.80      0.80      0.79       179


Performance metrics for K=5:

              precision    recall  f1-score   support

           0       0.79      0.90      0.84       105
           1       0.83      0.65      0.73        74

    accuracy                           0.80       179
   macro avg       0.81      0.78      0.78       179
weighted avg       0.80      0.80      0.79       179


Performance metrics for K=7:

              precision    recall  f1-score   support

           0       0.79      0.90      0.84       105
           1       0.83      0.65      0.73        74

    accuracy                           0.80       179
   macro avg       0.81      0.78   